## 4. Entraînement du modèle

In [ ]:
# Entraînement du modèle Morningstar
import tensorflow as tf
from Morningstar.workflows.training_workflow import TrainingWorkflow
import numpy as np

print(f"\n=== Préparation des données ===")
print(f"Taille du dataset original: {len(data)} échantillons")
print(f"Colonnes: {data.columns.tolist()}")

# Vérification des données avant conversion
print("\nExemple de données avant conversion:")
print(data.iloc[0])

# Configuration
class ColabConfig:
    def __init__(self):
        self.time_window = 50
        self.features = [col for col in data.columns if col not in ['datetime', 'pair']]
        self.epochs = 200
        self.batch_size = 1024
        self.dataset_path = 'full_dataset.parquet'

colab_config = ColabConfig()
print(f"\nConfiguration utilisée: {vars(colab_config)}")

# Conversion en dataset TensorFlow avec vérification
print("\n=== Conversion en dataset TensorFlow ===")
workflow = TrainingWorkflow(colab_config)
try:
    tf_dataset = workflow._prepare_dataset(data)
    
    # Vérification du dataset
    print("\nVérification du dataset TensorFlow:")
    sample = next(iter(tf_dataset))
    print(f"Type: {type(sample)}")
    print(f"Contenu: {sample}")
    
    dataset_size = tf.data.experimental.cardinality(tf_dataset).numpy()
    print(f"Taille du dataset TensorFlow: {dataset_size} batches")
    
    if dataset_size < 1:
        raise ValueError("Le dataset TensorFlow est vide après conversion")
        
except Exception as e:
    print(f"\nERREUR lors de la conversion: {str(e)}")
    print("\nDébogage - Tentative de conversion manuelle:")
    
    # Conversion manuelle de secours
    features = data[colab_config.features].values.astype(np.float32)
    targets = data['close'].values.astype(np.float32)
    
    print(f"Forme des features: {features.shape}")
    print(f"Forme des targets: {targets.shape}")
    
    # Création du dataset manuellement
    tf_dataset = tf.data.Dataset.from_tensor_slices((features, targets))
    tf_dataset = tf_dataset.batch(colab_config.batch_size)
    
    print("\nDataset créé manuellement avec succès")
    dataset_size = tf.data.experimental.cardinality(tf_dataset).numpy()
    print(f"Taille du dataset manuel: {dataset_size} batches")

# Division train/val
print("\n=== Division train/validation ===")
val_size = max(1, int(dataset_size * 0.2))
train_dataset = tf_dataset.skip(val_size)
val_dataset = tf_dataset.take(val_size)

print(f"Train batches: {tf.data.experimental.cardinality(train_dataset).numpy()}")
print(f"Val batches: {tf.data.experimental.cardinality(val_dataset).numpy()}")

# Construction du modèle
print("\n=== Construction du modèle ===")
with tf.distribute.MirroredStrategy().scope():
    input_shape = (colab_config.time_window, len(colab_config.features))
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Conv1D(128, 5, activation='swish')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LSTM(256, return_sequences=True)(x)
    x = tf.keras.layers.LSTM(128)(x)
    x = tf.keras.layers.Dense(64, activation='swish')(x)
    outputs = tf.keras.layers.Dense(1)(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='huber',
        metrics=['mae']
    )
model.summary()

# Callbacks
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss'),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss', restore_best_weights=True)
]

# Entraînement
print("\n=== Début de l'entraînement ===")
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=colab_config.epochs,
    callbacks=callbacks
)

## 5. Sauvegarde finale

In [ ]:
# Sauvegarde finale et export sur Google Drive
model.save('morningstar_pro.h5')
from google.colab import drive
drive.mount('/content/drive')
!cp morningstar_pro.h5 '/content/drive/MyDrive/Colab Data/'
print("Modèle sauvegardé avec succès")